In [1]:
import sys
sys.path.append("../code")
from dataset import build_dataset, build_dataloader
from config import DataArguments, TrainerArguments, ModelArguments

In [17]:
data_args = DataArguments()
training_args = TrainerArguments()
model_args = ModelArguments()

data_args.data_dir = "../data/"
data_args.seed = training_args.seed
data_args.max_seq_len = model_args.max_seq_len

data_args.init_pct = 0.01

# train_dataset, model_args.vocab_size, model_args.num_labels = build_dataset(data_args, "train")
# train_dataloader = build_dataloader(train_dataset, data_args)

In [19]:
dict(
    **data_args.to_dict(),
    **training_args.to_dict()
)

{'asset_dir': './assets/',
 'area2idx': 'area2idx.json',
 'data_dir': '../data/',
 'dataset_name': 'paperswithcode',
 'init_pct': 0.01,
 'balanced': False,
 'use_abstract': False,
 'use_task_id': False,
 'preprocessor': 'spacy',
 'spacy_model': 'en_core_web_trf',
 'batch_size': 64,
 'pin_memory': True,
 'seed': 42,
 'configuration_keys': 'configuration_keys.json',
 'active_learning': False,
 'increment_pct': 0.05,
 'approximation': 'mcdropout',
 'acquisition': 'lc',
 'output_dir': 'output/default',
 'overwrite_output_dir': False,
 'do_train': False,
 'do_valid': False,
 'do_test': False,
 'num_train_epochs': 10,
 'gradient_accumulation_steps': 1,
 'optimizer': 'adamw',
 'learning_rate': 2e-05,
 'warmup_steps': 100,
 'weight_decay': 0.01,
 'use_gpu': True,
 'project': 'Active Learning',
 'run_name': None}

In [11]:
data_args = DataArguments()

In [3]:
from trainer import NaiveTrainer

training_args.do_train = True

trainer = NaiveTrainer(
    training_args,
    model_args,
    training_dataset=train_dataloader if training_args.do_train else None,
    validation_dataset=valid_dataloader if training_args.do_valid else None,
    test_dataset=test_dataloader if training_args.do_test else None,
)

[11/21/2021 15:34:25] INFO - trainer: Bert was selected. Start Transformer setup.
[11/21/2021 15:34:25] INFO - trainer: Successfully setup transformer settings.


In [6]:
batch = next(iter(train_dataloader))
batch = {k: v.cuda() for k, v in batch.items()}
model = trainer.model

In [8]:
logits, prediction_class = model(batch)

In [13]:
from torch.nn import Softmax

sfx_logits = Softmax(dim=1)(logits)

In [46]:
import torch
torch.vstack([]).shape

TypeError: expected Tensor as element 1 in argument 0, but got list

In [25]:
from sklearn.metrics import roc_auc_score

label = batch["labels"].cpu().numpy()
logit = sfx_logits.detach().cpu().numpy()


In [30]:
roc_auc_score(label, logit, multi_class='ovo')

ValueError: Number of classes in y_true not equal to the number of columns in 'y_score'

In [50]:
torch.cat([batch["labels"].cpu() for _ in range(3)])

tensor([ 7, 10,  7, 14, 13, 12, 14, 14,  7,  3, 10,  8, 10, 12, 14,  7,  6, 10,
         6,  7, 10,  3, 15, 15, 10, 10,  8,  7, 12, 15, 10, 14,  4,  7, 15,  3,
         3, 15, 10,  7, 14, 13, 15, 10, 12,  3,  7, 12, 13,  7,  5, 11, 12, 15,
         7,  7, 12,  4,  7,  7,  8,  3,  7,  5,  7, 10,  7, 14, 13, 12, 14, 14,
         7,  3, 10,  8, 10, 12, 14,  7,  6, 10,  6,  7, 10,  3, 15, 15, 10, 10,
         8,  7, 12, 15, 10, 14,  4,  7, 15,  3,  3, 15, 10,  7, 14, 13, 15, 10,
        12,  3,  7, 12, 13,  7,  5, 11, 12, 15,  7,  7, 12,  4,  7,  7,  8,  3,
         7,  5,  7, 10,  7, 14, 13, 12, 14, 14,  7,  3, 10,  8, 10, 12, 14,  7,
         6, 10,  6,  7, 10,  3, 15, 15, 10, 10,  8,  7, 12, 15, 10, 14,  4,  7,
        15,  3,  3, 15, 10,  7, 14, 13, 15, 10, 12,  3,  7, 12, 13,  7,  5, 11,
        12, 15,  7,  7, 12,  4,  7,  7,  8,  3,  7,  5])

In [39]:
import numpy as np
label_sample = np.random.randint(0, 16, size=64)

In [40]:
roc_auc_score(label_sample, logit, multi_class='ovo')

0.4712041997354497

In [54]:
logit.argmax(axis=1).shape

(64,)

In [55]:
logit.shape

(64, 16)

In [61]:
training_args.__format__()

TypeError: descriptor '__format__' of 'object' object needs an argument

In [66]:
data_args.to_dict()


AttributeError: type object 'DataArguments' has no attribute 'to_dict'